<a href="https://colab.research.google.com/github/prashant-gulati/colab/blob/main/MCP_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Context Protocol (MCP)

### A Practical Guide to Building AI Tool Integrations

---

The Model Context Protocol is an open standard introduced by Anthropic that enables AI assistants to connect with external data sources and tools. This tutorial demonstrates the core concepts through working code examples.

**What you will learn:**

- How MCP enables communication between AI clients and tool servers
- The three primitives: Tools, Resources, and Prompts
- How to build a functional MCP server from scratch
- How clients discover and invoke server capabilities

---

## Contents

1. Understanding MCP Architecture
2. The JSON-RPC Protocol
3. Building an MCP Server
4. Building an MCP Client
5. Interactive Demonstration
6. Real-World Patterns

---
## 1. Understanding MCP Architecture

MCP follows a client-server model where AI applications (clients) connect to specialized servers that expose tools and data.

```
                    MCP Architecture
                    
    +-------------+          +-------------+
    |   Claude    |          |   GPT-4     |
    |  (Client)   |          |  (Client)   |
    +------+------+          +------+------+
           |                        |
           v                        v
    +------+------------------------+------+
    |           MCP Protocol               |
    |         (JSON-RPC 2.0)               |
    +------+--------+--------+-------------+
           |        |        |
           v        v        v
    +------+--+ +---+----+ +-+--------+
    | Weather | | GitHub | | Database |
    | Server  | | Server | |  Server  |
    +---------+ +--------+ +----------+
```

### Key Concepts

| Component | Role |
|-----------|------|
| Host | The application that embeds the AI (e.g., Claude Desktop) |
| Client | Component within the host that manages server connections |
| Server | Exposes tools, resources, and prompts to clients |
| Transport | Communication channel (stdio, HTTP, WebSocket) |

### Three Primitives

MCP servers can expose three types of capabilities:

| Primitive | Description | Control |
|-----------|-------------|--------|
| Tools | Functions the LLM can invoke | Model-controlled |
| Resources | Read-only data (files, API responses) | Application-controlled |
| Prompts | Reusable prompt templates | User-controlled |

---
## 2. The JSON-RPC Protocol

MCP uses JSON-RPC 2.0 as its wire format. Every message follows a standard structure.

### Request Format

```json
{
  "jsonrpc": "2.0",
  "id": 1,
  "method": "tools/call",
  "params": {
    "name": "get_weather",
    "arguments": {"city": "Tokyo"}
  }
}
```

### Response Format

```json
{
  "jsonrpc": "2.0",
  "id": 1,
  "result": {
    "content": [
      {"type": "text", "text": "Weather in Tokyo: 72F, Clear"}
    ]
  }
}
```

### Core Methods

| Method | Description |
|--------|-------------|
| initialize | Establish connection and exchange capabilities |
| tools/list | Retrieve available tools from server |
| tools/call | Execute a specific tool |
| resources/list | List available resources |
| resources/read | Read a specific resource |
| prompts/list | List available prompt templates |
| prompts/get | Retrieve a specific prompt |

---
## 3. Building an MCP Server

Let us implement a complete MCP server from scratch. This implementation follows the official protocol specification.

In [ ]:
import json
import asyncio
from dataclasses import dataclass, field, asdict
from typing import Callable, Optional, List, Dict, Any
from datetime import datetime
from IPython.display import display, HTML

print("Imports complete.")

Imports complete.


### 3.1 Define Data Structures

These classes represent the core MCP protocol types.

In [ ]:
@dataclass
class Tool:
    """Represents a callable tool in the MCP server."""
    name: str
    description: str
    input_schema: Dict[str, Any]
    handler: Callable


@dataclass
class Resource:
    """Represents a readable resource in the MCP server."""
    uri: str
    name: str
    description: str
    mime_type: str = "text/plain"


@dataclass
class Prompt:
    """Represents a reusable prompt template."""
    name: str
    description: str
    arguments: List[Dict[str, str]] = field(default_factory=list)


@dataclass
class ServerCapabilities:
    """Capabilities exposed by the MCP server."""
    tools: bool = True
    resources: bool = True
    prompts: bool = True


@dataclass
class ServerInfo:
    """Server identification and version."""
    name: str
    version: str


print("Data structures defined.")
print("")
print("Classes:")
print("  - Tool: Callable function with schema")
print("  - Resource: Read-only data source")
print("  - Prompt: Reusable prompt template")
print("  - ServerCapabilities: What the server supports")
print("  - ServerInfo: Server metadata")

Data structures defined.

Classes:
  - Tool: Callable function with schema
  - Resource: Read-only data source
  - Prompt: Reusable prompt template
  - ServerCapabilities: What the server supports
  - ServerInfo: Server metadata


### 3.2 Implement the MCP Server

The server handles JSON-RPC requests and routes them to the appropriate handlers.

In [ ]:
class MCPServer:
    """
    A minimal MCP server implementation.

    Handles the core protocol methods:
    - initialize: Exchange capabilities
    - tools/list: Return available tools
    - tools/call: Execute a tool
    - resources/list: Return available resources
    - resources/read: Read a resource
    - prompts/list: Return available prompts
    - prompts/get: Get a prompt template
    """

    PROTOCOL_VERSION = "2024-11-05"

    def __init__(self, name: str, version: str = "1.0.0"):
        self.info = ServerInfo(name=name, version=version)
        self.capabilities = ServerCapabilities()
        self.tools: Dict[str, Tool] = {}
        self.resources: Dict[str, Resource] = {}
        self.prompts: Dict[str, Prompt] = {}
        self._resource_handlers: Dict[str, Callable] = {}
        self._prompt_handlers: Dict[str, Callable] = {}
        self._initialized = False

    def register_tool(
        self,
        name: str,
        description: str,
        input_schema: Dict[str, Any],
        handler: Callable
    ):
        """Register a new tool with the server."""
        self.tools[name] = Tool(
            name=name,
            description=description,
            input_schema=input_schema,
            handler=handler
        )

    def register_resource(
        self,
        uri: str,
        name: str,
        description: str,
        handler: Callable,
        mime_type: str = "text/plain"
    ):
        """Register a new resource with the server."""
        self.resources[uri] = Resource(
            uri=uri,
            name=name,
            description=description,
            mime_type=mime_type
        )
        self._resource_handlers[uri] = handler

    def register_prompt(
        self,
        name: str,
        description: str,
        arguments: List[Dict[str, str]],
        handler: Callable
    ):
        """Register a new prompt template with the server."""
        self.prompts[name] = Prompt(
            name=name,
            description=description,
            arguments=arguments
        )
        self._prompt_handlers[name] = handler

    def handle_request(self, request: Dict[str, Any]) -> Dict[str, Any]:
        """Process a JSON-RPC request and return a response."""
        request_id = request.get("id")
        method = request.get("method", "")
        params = request.get("params", {})

        try:
            # Route to appropriate handler
            if method == "initialize":
                result = self._handle_initialize(params)
            elif method == "tools/list":
                result = self._handle_tools_list()
            elif method == "tools/call":
                result = self._handle_tools_call(params)
            elif method == "resources/list":
                result = self._handle_resources_list()
            elif method == "resources/read":
                result = self._handle_resources_read(params)
            elif method == "prompts/list":
                result = self._handle_prompts_list()
            elif method == "prompts/get":
                result = self._handle_prompts_get(params)
            else:
                return self._error_response(request_id, -32601, f"Method not found: {method}")

            return {
                "jsonrpc": "2.0",
                "id": request_id,
                "result": result
            }
        except Exception as e:
            return self._error_response(request_id, -32000, str(e))

    def _handle_initialize(self, params: Dict) -> Dict:
        """Handle the initialize request."""
        self._initialized = True
        return {
            "protocolVersion": self.PROTOCOL_VERSION,
            "capabilities": {
                "tools": {} if self.capabilities.tools else None,
                "resources": {} if self.capabilities.resources else None,
                "prompts": {} if self.capabilities.prompts else None,
            },
            "serverInfo": {
                "name": self.info.name,
                "version": self.info.version
            }
        }

    def _handle_tools_list(self) -> Dict:
        """Return list of available tools."""
        return {
            "tools": [
                {
                    "name": tool.name,
                    "description": tool.description,
                    "inputSchema": tool.input_schema
                }
                for tool in self.tools.values()
            ]
        }

    def _handle_tools_call(self, params: Dict) -> Dict:
        """Execute a tool and return the result."""
        tool_name = params.get("name")
        arguments = params.get("arguments", {})

        if tool_name not in self.tools:
            raise ValueError(f"Unknown tool: {tool_name}")

        tool = self.tools[tool_name]
        result = tool.handler(**arguments)

        return {
            "content": [
                {"type": "text", "text": str(result)}
            ]
        }

    def _handle_resources_list(self) -> Dict:
        """Return list of available resources."""
        return {
            "resources": [
                {
                    "uri": res.uri,
                    "name": res.name,
                    "description": res.description,
                    "mimeType": res.mime_type
                }
                for res in self.resources.values()
            ]
        }

    def _handle_resources_read(self, params: Dict) -> Dict:
        """Read a resource and return its contents."""
        uri = params.get("uri")

        if uri not in self.resources:
            raise ValueError(f"Unknown resource: {uri}")

        handler = self._resource_handlers[uri]
        content = handler()
        resource = self.resources[uri]

        return {
            "contents": [
                {
                    "uri": uri,
                    "mimeType": resource.mime_type,
                    "text": content
                }
            ]
        }

    def _handle_prompts_list(self) -> Dict:
        """Return list of available prompts."""
        return {
            "prompts": [
                {
                    "name": prompt.name,
                    "description": prompt.description,
                    "arguments": prompt.arguments
                }
                for prompt in self.prompts.values()
            ]
        }

    def _handle_prompts_get(self, params: Dict) -> Dict:
        """Get a prompt template with arguments filled in."""
        name = params.get("name")
        arguments = params.get("arguments", {})

        if name not in self.prompts:
            raise ValueError(f"Unknown prompt: {name}")

        handler = self._prompt_handlers[name]
        messages = handler(**arguments)

        return {
            "messages": messages
        }

    def _error_response(self, request_id: Any, code: int, message: str) -> Dict:
        """Create a JSON-RPC error response."""
        return {
            "jsonrpc": "2.0",
            "id": request_id,
            "error": {
                "code": code,
                "message": message
            }
        }


print("MCPServer class defined.")
print("")
print("Methods:")
print("  - register_tool(): Add a callable tool")
print("  - register_resource(): Add a readable resource")
print("  - register_prompt(): Add a prompt template")
print("  - handle_request(): Process JSON-RPC requests")

MCPServer class defined.

Methods:
  - register_tool(): Add a callable tool
  - register_resource(): Add a readable resource
  - register_prompt(): Add a prompt template
  - handle_request(): Process JSON-RPC requests


### 3.3 Register Tools, Resources, and Prompts

Now we register capabilities with our server.

In [ ]:
# Create the server instance
server = MCPServer(name="demo-server", version="1.0.0")


# --- TOOLS ---

def get_weather(city: str) -> str:
    """Simulated weather API."""
    weather_data = {
        "tokyo": (72, "Clear", 55),
        "london": (58, "Rainy", 85),
        "new york": (68, "Partly Cloudy", 60),
        "paris": (64, "Overcast", 70),
        "mumbai": (88, "Humid", 80),
    }
    temp, condition, humidity = weather_data.get(city.lower(), (70, "Unknown", 50))
    return f"Weather in {city}: {temp}F, {condition}, Humidity: {humidity}%"


def calculate(expression: str) -> str:
    """Safe math evaluator."""
    allowed = set("0123456789+-*/.() ")
    if not all(c in allowed for c in expression):
        return "Error: Invalid characters"
    try:
        result = eval(expression, {"__builtins__": {}}, {})
        return f"{expression} = {result}"
    except Exception as e:
        return f"Error: {e}"


def get_time(timezone: str = "UTC") -> str:
    """Return current time."""
    return f"Current time ({timezone}): {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"


# Register tools
server.register_tool(
    name="get_weather",
    description="Get current weather for a city",
    input_schema={
        "type": "object",
        "properties": {
            "city": {"type": "string", "description": "City name"}
        },
        "required": ["city"]
    },
    handler=get_weather
)

server.register_tool(
    name="calculate",
    description="Evaluate a mathematical expression",
    input_schema={
        "type": "object",
        "properties": {
            "expression": {"type": "string", "description": "Math expression"}
        },
        "required": ["expression"]
    },
    handler=calculate
)

server.register_tool(
    name="get_time",
    description="Get the current date and time",
    input_schema={
        "type": "object",
        "properties": {
            "timezone": {"type": "string", "description": "Timezone", "default": "UTC"}
        }
    },
    handler=get_time
)


# --- RESOURCES ---

def get_config_data():
    return json.dumps({"version": "1.0", "debug": False, "max_tokens": 4096}, indent=2)


def get_readme_data():
    return "# Demo MCP Server\n\nThis server demonstrates the Model Context Protocol."


server.register_resource(
    uri="config://app/settings",
    name="Application Settings",
    description="Current application configuration",
    handler=get_config_data,
    mime_type="application/json"
)

server.register_resource(
    uri="file://docs/readme",
    name="README",
    description="Project documentation",
    handler=get_readme_data,
    mime_type="text/markdown"
)


# --- PROMPTS ---

def code_review_prompt(language: str, code: str):
    return [
        {
            "role": "user",
            "content": {
                "type": "text",
                "text": f"Please review this {language} code:\n\n```{language}\n{code}\n```"
            }
        }
    ]


def summarize_prompt(text: str, style: str = "concise"):
    return [
        {
            "role": "user",
            "content": {
                "type": "text",
                "text": f"Summarize the following text in a {style} manner:\n\n{text}"
            }
        }
    ]


server.register_prompt(
    name="code_review",
    description="Generate a code review prompt",
    arguments=[
        {"name": "language", "description": "Programming language", "required": True},
        {"name": "code", "description": "Code to review", "required": True}
    ],
    handler=code_review_prompt
)

server.register_prompt(
    name="summarize",
    description="Generate a summarization prompt",
    arguments=[
        {"name": "text", "description": "Text to summarize", "required": True},
        {"name": "style", "description": "Summary style", "required": False}
    ],
    handler=summarize_prompt
)


print("Server configured.")
print("")
print(f"Tools registered: {len(server.tools)}")
for name in server.tools:
    print(f"  - {name}")
print("")
print(f"Resources registered: {len(server.resources)}")
for uri in server.resources:
    print(f"  - {uri}")
print("")
print(f"Prompts registered: {len(server.prompts)}")
for name in server.prompts:
    print(f"  - {name}")

Server configured.

Tools registered: 3
  - get_weather
  - calculate
  - get_time

Resources registered: 2
  - config://app/settings
  - file://docs/readme

Prompts registered: 2
  - code_review
  - summarize


---
## 4. Building an MCP Client

The client sends JSON-RPC requests and processes responses.

In [ ]:
class MCPClient:
    """
    A minimal MCP client implementation.

    In production, this would communicate over stdio or HTTP.
    For demonstration, we connect directly to a server instance.
    """

    def __init__(self, server: MCPServer):
        self.server = server
        self._request_id = 0
        self._server_info = None
        self._capabilities = None

    def _next_id(self) -> int:
        self._request_id += 1
        return self._request_id

    def _send_request(self, method: str, params: Dict = None) -> Dict:
        """Send a JSON-RPC request to the server."""
        request = {
            "jsonrpc": "2.0",
            "id": self._next_id(),
            "method": method,
            "params": params or {}
        }
        return self.server.handle_request(request)

    def initialize(self) -> Dict:
        """Initialize connection with the server."""
        response = self._send_request("initialize", {
            "protocolVersion": "2024-11-05",
            "capabilities": {},
            "clientInfo": {
                "name": "demo-client",
                "version": "1.0.0"
            }
        })
        if "result" in response:
            self._server_info = response["result"].get("serverInfo")
            self._capabilities = response["result"].get("capabilities")
        return response

    def list_tools(self) -> List[Dict]:
        """Get available tools from the server."""
        response = self._send_request("tools/list")
        if "result" in response:
            return response["result"].get("tools", [])
        return []

    def call_tool(self, name: str, arguments: Dict = None) -> Dict:
        """Execute a tool on the server."""
        return self._send_request("tools/call", {
            "name": name,
            "arguments": arguments or {}
        })

    def list_resources(self) -> List[Dict]:
        """Get available resources from the server."""
        response = self._send_request("resources/list")
        if "result" in response:
            return response["result"].get("resources", [])
        return []

    def read_resource(self, uri: str) -> Dict:
        """Read a resource from the server."""
        return self._send_request("resources/read", {"uri": uri})

    def list_prompts(self) -> List[Dict]:
        """Get available prompts from the server."""
        response = self._send_request("prompts/list")
        if "result" in response:
            return response["result"].get("prompts", [])
        return []

    def get_prompt(self, name: str, arguments: Dict = None) -> Dict:
        """Get a prompt template from the server."""
        return self._send_request("prompts/get", {
            "name": name,
            "arguments": arguments or {}
        })


# Create client connected to our server
client = MCPClient(server)

print("MCPClient class defined and instantiated.")

MCPClient class defined and instantiated.


---
## 5. Interactive Demonstration

Let us walk through the complete MCP interaction flow.

In [ ]:
def render_mcp_interaction(title: str, request: Dict, response: Dict):
    """Render an MCP interaction in a clean format."""
    html = """
    <style>
        .mcp-container {
            font-family: -apple-system, BlinkMacSystemFont, 'SF Pro Text', 'Segoe UI', Roboto, Helvetica, Arial, sans-serif;
            max-width: 720px;
            margin: 16px auto;
            font-size: 13px;
            line-height: 1.5;
            color: #1d1d1f;
        }
        .mcp-title {
            font-size: 14px;
            font-weight: 600;
            color: #1d1d1f;
            margin-bottom: 12px;
            padding-bottom: 8px;
            border-bottom: 1px solid #d2d2d7;
        }
        .mcp-section {
            margin: 10px 0;
        }
        .mcp-label {
            font-size: 10px;
            font-weight: 600;
            color: #86868b;
            text-transform: uppercase;
            letter-spacing: 0.5px;
            margin-bottom: 4px;
        }
        .mcp-code {
            font-family: 'SF Mono', Monaco, Consolas, monospace;
            font-size: 11px;
            background: #f5f5f7;
            border: 1px solid #d2d2d7;
            border-radius: 6px;
            padding: 10px 12px;
            overflow-x: auto;
            white-space: pre-wrap;
            word-break: break-all;
        }
        .mcp-request .mcp-code {
            border-left: 3px solid #0066cc;
        }
        .mcp-response .mcp-code {
            border-left: 3px solid #34c759;
        }
    </style>
    """

    html += f"""
    <div class="mcp-container">
        <div class="mcp-title">{title}</div>
        <div class="mcp-section mcp-request">
            <div class="mcp-label">Request</div>
            <div class="mcp-code">{json.dumps(request, indent=2)}</div>
        </div>
        <div class="mcp-section mcp-response">
            <div class="mcp-label">Response</div>
            <div class="mcp-code">{json.dumps(response, indent=2)}</div>
        </div>
    </div>
    """
    display(HTML(html))

### 5.1 Initialize Connection

In [ ]:
init_request = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "initialize",
    "params": {
        "protocolVersion": "2024-11-05",
        "clientInfo": {"name": "demo-client", "version": "1.0.0"}
    }
}

init_response = client.initialize()
render_mcp_interaction("Initialize Connection", init_request, init_response)

### 5.2 Discover Tools

In [ ]:
tools_request = {
    "jsonrpc": "2.0",
    "id": 2,
    "method": "tools/list",
    "params": {}
}

tools = client.list_tools()
tools_response = {"jsonrpc": "2.0", "id": 2, "result": {"tools": tools}}
render_mcp_interaction("List Available Tools", tools_request, tools_response)

### 5.3 Call a Tool

In [ ]:
weather_request = {
    "jsonrpc": "2.0",
    "id": 3,
    "method": "tools/call",
    "params": {
        "name": "get_weather",
        "arguments": {"city": "Tokyo"}
    }
}

weather_response = client.call_tool("get_weather", {"city": "Tokyo"})
render_mcp_interaction("Call Weather Tool", weather_request, weather_response)

In [ ]:
calc_request = {
    "jsonrpc": "2.0",
    "id": 4,
    "method": "tools/call",
    "params": {
        "name": "calculate",
        "arguments": {"expression": "(100 + 50) * 2"}
    }
}

calc_response = client.call_tool("calculate", {"expression": "(100 + 50) * 2"})
render_mcp_interaction("Call Calculator Tool", calc_request, calc_response)

### 5.4 Read Resources

In [ ]:
resources_request = {
    "jsonrpc": "2.0",
    "id": 5,
    "method": "resources/list",
    "params": {}
}

resources = client.list_resources()
resources_response = {"jsonrpc": "2.0", "id": 5, "result": {"resources": resources}}
render_mcp_interaction("List Available Resources", resources_request, resources_response)

In [ ]:
read_request = {
    "jsonrpc": "2.0",
    "id": 6,
    "method": "resources/read",
    "params": {"uri": "config://app/settings"}
}

read_response = client.read_resource("config://app/settings")
render_mcp_interaction("Read Configuration Resource", read_request, read_response)

### 5.5 Get Prompts

In [ ]:
prompts_request = {
    "jsonrpc": "2.0",
    "id": 7,
    "method": "prompts/list",
    "params": {}
}

prompts = client.list_prompts()
prompts_response = {"jsonrpc": "2.0", "id": 7, "result": {"prompts": prompts}}
render_mcp_interaction("List Available Prompts", prompts_request, prompts_response)

In [ ]:
prompt_request = {
    "jsonrpc": "2.0",
    "id": 8,
    "method": "prompts/get",
    "params": {
        "name": "code_review",
        "arguments": {
            "language": "python",
            "code": "def add(a, b): return a + b"
        }
    }
}

prompt_response = client.get_prompt("code_review", {
    "language": "python",
    "code": "def add(a, b): return a + b"
})
render_mcp_interaction("Get Code Review Prompt", prompt_request, prompt_response)

### 5.6 Interactive Tool Explorer

In [ ]:
def render_tool_explorer(client: MCPClient):
    """Render an interactive tool explorer."""
    tools = client.list_tools()

    html = """
    <style>
        .explorer {
            font-family: -apple-system, BlinkMacSystemFont, 'SF Pro Text', 'Segoe UI', Roboto, Helvetica, Arial, sans-serif;
            max-width: 720px;
            margin: 20px auto;
            color: #1d1d1f;
        }
        .explorer-header {
            font-size: 15px;
            font-weight: 600;
            margin-bottom: 16px;
            padding-bottom: 10px;
            border-bottom: 1px solid #d2d2d7;
        }
        .tool-card {
            background: #fbfbfd;
            border: 1px solid #d2d2d7;
            border-radius: 8px;
            padding: 14px 16px;
            margin-bottom: 10px;
        }
        .tool-name {
            font-family: 'SF Mono', Monaco, Consolas, monospace;
            font-size: 13px;
            font-weight: 600;
            color: #0066cc;
        }
        .tool-desc {
            font-size: 12px;
            color: #6e6e73;
            margin-top: 4px;
        }
        .tool-params {
            font-size: 11px;
            color: #86868b;
            margin-top: 8px;
            font-family: 'SF Mono', Monaco, Consolas, monospace;
        }
    </style>
    <div class="explorer">
        <div class="explorer-header">Available Tools</div>
    """

    for tool in tools:
        props = tool.get("inputSchema", {}).get("properties", {})
        params = ", ".join([f"{k}: {v.get('type', 'any')}" for k, v in props.items()])

        html += f"""
        <div class="tool-card">
            <div class="tool-name">{tool['name']}</div>
            <div class="tool-desc">{tool['description']}</div>
            <div class="tool-params">Parameters: {params if params else 'none'}</div>
        </div>
        """

    html += "</div>"
    display(HTML(html))


render_tool_explorer(client)

### 5.7 Try Your Own Tool Calls

In [ ]:
# Try calling tools interactively

print("Enter a tool name and arguments to test.")
print("")
print("Available tools: get_weather, calculate, get_time")
print("")

tool_name = input("Tool name: ").strip()

if tool_name == "get_weather":
    city = input("City: ").strip()
    response = client.call_tool("get_weather", {"city": city})

elif tool_name == "calculate":
    expr = input("Expression: ").strip()
    response = client.call_tool("calculate", {"expression": expr})

elif tool_name == "get_time":
    response = client.call_tool("get_time", {})

else:
    response = {"error": f"Unknown tool: {tool_name}"}

print("")
print("Response:")
print(json.dumps(response, indent=2))

Enter a tool name and arguments to test.

Available tools: get_weather, calculate, get_time

Tool name: get_weather
City: Pune

Response:
{
  "jsonrpc": "2.0",
  "id": 10,
  "result": {
    "content": [
      {
        "type": "text",
        "text": "Weather in Pune: 70F, Unknown, Humidity: 50%"
      }
    ]
  }
}


---
## 6. Real-World Patterns

### Transport Mechanisms

In production, MCP uses these transport mechanisms:

| Transport | Use Case |
|-----------|----------|
| stdio | Local servers (most common for desktop apps) |
| HTTP + SSE | Remote servers, web applications |
| WebSocket | Real-time bidirectional communication |

### Server Configuration for Claude Desktop

To use an MCP server with Claude Desktop, add it to `claude_desktop_config.json`:

```json
{
  "mcpServers": {
    "my-server": {
      "command": "npx",
      "args": ["-y", "my-mcp-server"]
    }
  }
}
```

### Best Practices

1. Never write to stdout in stdio-based servers (use stderr for logging)
2. Validate all input parameters with schemas
3. Return structured error messages
4. Keep tool descriptions clear and concise
5. Use appropriate MIME types for resources

---
## Summary

This tutorial demonstrated the core concepts of the Model Context Protocol:

1. **Architecture**: Client-server model using JSON-RPC 2.0
2. **Three Primitives**: Tools (actions), Resources (data), Prompts (templates)
3. **Server Implementation**: Registering and handling capabilities
4. **Client Implementation**: Discovery and invocation patterns

The MCP standard enables AI assistants to securely connect with external systems through a unified protocol, replacing custom integrations with a standardized approach.

### References

- Official Documentation: https://modelcontextprotocol.io
- TypeScript SDK: https://github.com/modelcontextprotocol/typescript-sdk
- Python SDK: https://github.com/modelcontextprotocol/python-sdk
- Example Servers: https://github.com/modelcontextprotocol/servers

---